<a href="https://colab.research.google.com/github/Nour-chargui/PES-Challenge/blob/main/Potato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install nltk
!pip install sympy

In [11]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


37 patterns
11 tags: ['education', 'fun_activities', 'funny', 'games', 'goodbye', 'greeting', 'learning_subjects', 'learning_tools', 'math_game', 'space_information', 'thanks']
69 unique stemmed words: ["'m", "'s", ',', 'a', 'about', 'activ', 'ani', 'anyon', 'are', 'bore', 'bye', 'can', 'capit', 'cool', 'day', 'differ', 'do', 'fact', 'for', 'fun', 'funni', 'game', 'give', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'in', 'inform', 'interest', 'is', 'joke', 'know', 'later', 'learn', 'let', 'lot', 'math', 'me', 'my', 'of', 'outer', 'play', 'see', 'skill', 'some', 'someth', 'space', 'studi', 'subject', 'suggest', 'teach', 'tell', 'test', 'thank', 'that', 'the', 'there', 'to', 'tool', 'tunisia', 'want', 'what', 'which', 'you']
69 11
Epoch [100/1000], Loss: 2.0240
Epoch [200/1000], Loss: 2.3602
Epoch [300/1000], Loss: 1.3884
Epoch [400/1000], Loss: 1.5803
Epoch [500/1000], Loss: 1.1613
Epoch [600/1000], Loss: 1.7242
Epoch [700/1000], Loss: 1.1017
Epoch [800/1000], Loss: 2.03

In [16]:
import random
import json
import torch
from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence.lower() == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    #print(f"Predicted Tag: {tag}, Confidence: {prob.item()}")

    if prob.item() > 0.4:  # Adjust the confidence threshold as needed
        for intent in intents['intents']:
            if tag == intent["tag"]:
                if tag == "education":
                    print(f"{bot_name}: {intent['responses'][0]}")  # Select the educational response
                else:
                    print(f"{bot_name}: {random.choice(intent['responses'])}")
                break
    else:
        print(f"{bot_name}: I do not understand...")


Let's chat! (type 'quit' to exit)
You: Hi
Predicted Tag: greeting, Confidence: 0.8304205536842346
Sam: Hi there, how can I make your day awesome?
You: Tell me fun joke
Predicted Tag: thanks, Confidence: 0.2560621500015259
Sam: I do not understand...
You: What is the capital of Tunisia?
Predicted Tag: education, Confidence: 0.6214727163314819
Sam: Sure! The capital of Tunisia is Tunis. Did you know that Tunisia is known for its beautiful beaches and ancient ruins?
You: Tell me a joke!
Predicted Tag: funny, Confidence: 0.42743077874183655
Sam: Why did the sun go to school? To get a little brighter!
You: quit
